In [ ]:
#this notebook is use to collect the first datas

In [21]:
import pandas as pd
import requests
import unicodedata
import re
import googlemaps
from bs4 import BeautifulSoup

import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# neighborhood São Paulo

In [17]:
url_neighborhood = 'https://www.estadosecapitaisdobrasil.com/listas/lista-dos-bairros-de-sao-paulo/'

In [183]:
#find all neighborhood in São Paulo
response = requests.get(url_neighborhood)
html = response.content
soup = BeautifulSoup(html)
neighborhood_sp = []
for neighb in soup.find_all('li')[24:]:
    neighborhood_sp.append(unicodedata.normalize('NFKD', neighb.text).encode('ASCII', 'ignore').decode('utf-8').strip().lower().replace(' ','+'))

In [185]:
print(neighborhood_sp)

['agua+rasa', 'alto+de+pinheiros', 'anhanguera', 'aricanduva', 'artur+alvim', 'barra+funda', 'bela+vista', 'belem', 'bom+retiro', 'brasilandia', 'butanta', 'cachoeirinha', 'cambuci', 'campo+belo', 'campo+grande', 'campo+limpo', 'cangaiba', 'capao+redondo', 'carrao', 'casa+verde', 'cidade+ademar', 'cidade+dutra', 'cidade+lider', 'cidade+lider', 'cidade+tiradentes', 'consolacao', 'cursino', 'ermelino+matarazzo', 'freguesia+do+o', 'grajau', 'guaianases', 'iguatemi', 'ipiranga', 'itaim+bibi', 'itaim+paulista', 'itaquera', 'jabaquara', 'jacana', 'jaguara', 'jaguare', 'jaragua', 'jardim+angela', 'jardim+helena', 'jardim+paulista', 'jardim+sao+luis', 'lapa', 'liberdade', 'limao', 'mandaqui', 'marsilac', 'moema', 'mooca', 'morumbi', 'parelheiros', 'pari', 'parque+do+carmo', 'penha', 'perdizes', 'pinheiros', 'ponte+rasa', 'raposo+tavares', 'republica', 'rio+pequeno', 'sacoma', 'santa+cecilia', 'santana', 'santo+amaro', 'sao+domingos', 'sao+lucas', 'sao+mateus', 'sao+miguel+paulista', 'sao+rafae

# streets São Paulo

In [188]:
#this code get the streets from neighborhood selected previously
df_street = pd.DataFrame()
for neighborhood in neighborhood_sp:
    for pag in range(1,10):
        try:
            url = fr'https://cep.guiamais.com.br/busca/{neighborhood}-sao+paulo-sp?page={pag}'
            response = requests.get(url)
            html = response.content
            soup = BeautifulSoup(html)
            table = soup.find_all('table')
            data = pd.read_html(str(table))[0]
            df_street = df_street.append(data, ignore_index=True)
        except:
            pass

In [192]:
df_street.head()

,LOGRADOURO,BAIRRO,CIDADE/ESTADO,BAIRRO.1,CEP
0,Rua Areas,Água Rasa,"São Paulo, SP","Água Rasa, São Paulo, SP",03179-020
1,Rua Barão de Penedo,Água Rasa,"São Paulo, SP","Água Rasa, São Paulo, SP",03179-070
2,Rua Barão de Penedo 107,Água Rasa,"São Paulo, SP","Água Rasa, São Paulo, SP",03175-901
3,Rua Barão de Penedo 107,Água Rasa,"São Paulo, SP","Água Rasa, São Paulo, SP",03179-900
4,Rua Capitão João Rosa da Cruz,Água Rasa,"São Paulo, SP","Água Rasa, São Paulo, SP",03179-080


In [195]:
#saving the dataframe 
df_street.to_csv('../data/raw-data/df_street.csv', index = False)

# Quinto Andar - apartment for rent

## transforming street to search apartment

In [197]:
#search works better with the pattern street-street-neighborhoodapartment

In [204]:
def transform_research_neighborhood(row):
    row = unidecode.unidecode(row).lower()
    row = row.replace(' ','-')
    return row

In [205]:
def transform_street(row):
    row = unidecode.unidecode(row).lower()
    row = row.replace('- ate ','')
    row = row.replace(' - lado impar','')
    row = row.replace(' - lado par','')
    row = row.replace(' ','-')
    return row

In [206]:
def clean_street_variable(row):
    row = row.replace('--de-','')
    row = row.split('-a-')[0]
    row = row.split('-ao-')[0]   
    row = row.replace('/','-')
    return row

In [207]:
df_street['BAIRRO'] = df_street['BAIRRO'].apply(transform_research_neighborhood)
df_street['LOGRADOURO'] = df_street['LOGRADOURO'].apply(transform_street)

In [208]:
qa_streets_search = df_street['LOGRADOURO'] + '-' + df_street['BAIRRO']

In [209]:
qa_streets_search = qa_streets_search.apply(clean_street_variable)

In [210]:
#return a list with streets to search in quinto andar website
list_streets = []
for rua in range(qa_streets_search.size):
    list_streets.append(qa_streets_search[rua])

## scraping id aparaments using photos name

In [ ]:
#the website contain a card with basic descriptions. 
#The image in card contains a ID that can be used to scraping more data in the future

In [ ]:
list_apartment_photo = []

for streets in list_streets:
    url = f'https://www.quintoandar.com.br/alugar/imovel/{streets}-sao-paulo-sp-brasil/'
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html)
    
    #advertising [2], [10] ignore them
    for tag_card in [x for x in range(18) if x != 2 and x != 10]:
        try:
            cards = soup.find_all('div', attrs={'class':'sc-1qwl1yl-0 igVsBW'})[tag_card]
            
            photo = cards.find('div' ,attrs={'class':'sc-1nwwxw0-0 drvPOq'}).img['src']


            apartment = {  
                            'photo': photo
                         }

            list_apartment_photo.append(apartment)
        except:
            pass

In [ ]:
df_qa_id = pd.DataFrame(list_apartment_photo)

## extract id

In [ ]:
def pick_id(row):
    row = row.split('/img/med/')[1]
    return row.split('-')[0]

In [ ]:
df_qa_id['ID'] = df_qa_id['photo'].apply(pick_id)
df_qa_id = df_qa_id['ID']
df_qa_id = df_qa_id.drop_duplicates()

In [ ]:
##saving the dataframe
df_qa_id.to_csv('../data/raw-data/df_qa_id.csv', index = False)

## scraping rent apartment information

In [ ]:
#scrap from quinto andar website, cointains value and features from apartment
#it uses the ID collected previously to obtain more information

In [ ]:
df_qa_id = pd.read_csv(r'data/df_qa_id.csv')

In [ ]:
list_apartment = []

for cod in df_qa_id['ID']:
    url = f'https://www.quintoandar.com.br/imovel/{cod}'
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html)
    

    
    try:
        endereco = re.findall(r'address:(.*)availabilityType',soup.text)
        endereco = endereco[0].replace('},"e"],','')
        endereco = endereco.replace('[{','')
        endereco = endereco.split(',')            
        
        
        
        
        tipo_imovel = soup.find('h1').text
        logradouro = soup.find('p',attrs={'class':'sc-bdVaJa gZyMJB'}).text.split(',')[0]     
        bairro = soup.find('p',attrs={'class':'sc-bdVaJa gZyMJB'}).text.split(',')[1]                
        cidade = soup.find('p',attrs={'class':'sc-bdVaJa gZyMJB'}).text.split(',')[2]         
        metragem = soup.find_all('span',attrs={'class':'sc-bdVaJa eNFcTE'})[0].text
        quartos = soup.find_all('span',attrs={'class':'sc-bdVaJa eNFcTE'})[1].text
        banheiros = soup.find_all('span',attrs={'class':'sc-bdVaJa eNFcTE'})[2].text
        vaga = soup.find_all('span',attrs={'class':'sc-bdVaJa eNFcTE'})[3].text
        andar = soup.find_all('span',attrs={'class':'sc-bdVaJa eNFcTE'})[4].text
        mobilia = soup.find_all('span',attrs={'class':'sc-bdVaJa eNFcTE'})[6].text
        aluguel = soup.find_all('span',attrs={'class':'ek9a7p-0 djWxJW sc-11qijje-0 iRCHiJ'})[0].text
        condominio = soup.find_all('span',attrs={'class':'ek9a7p-0 djWxJW sc-11qijje-0 bkKyCN'})[0].text
        iptu = soup.find_all('span',attrs={'class':'ek9a7p-0 djWxJW sc-11qijje-0 bkKyCN'})[1].text
        preco_total = soup.find_all('span',attrs={'class':'ek9a7p-0 djWxJW sc-11qijje-0 iRCHiJ'})[1].text
    
    
    
        apartment = { 
                        'codigo' : cod,
                        'endereco' : endereco,
                        'tipo_imovel' : tipo_imovel,
                        'logradouro' : logradouro,
                        'bairro' : bairro,
                        'cidade' : cidade,
                        'metragem' : metragem,
                        'quartos': quartos,
                        'banheiros': banheiros,
                        'vaga': vaga,
                        'andar' : andar,
                        'mobilia' : mobilia,
                        'aluguel': aluguel,
                        'condominio' : condominio,
                        'iptu' : iptu,
                        'preco_total' : preco_total,
                     }

        list_apartment.append(apartment)   
            
            
    except:
        pass

In [ ]:
df_qa_apartment_raw = pd.DataFrame(list_apartment)

In [ ]:
##saving the dataframe
df_qa_apartment_raw.to_csv('../data/raw-data/df_qa_apartment_raw.csv', index = False)

# Loft - apartment for sale

## scraping loft id 

In [2]:
#loft uses a code for every neighborhood where they have apartments
neighborhood_loft = [97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 131, 132, 133, 134, 135, 136, 137, 138, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 157, 159, 160, 161, 162, 163, 165, 166, 167, 168, 169, 171, 172, 173, 176, 177, 178, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 193, 194, 196, 197, 198, 201, 203, 204, 207, 208]

In [ ]:
#this code apply for each neighborhood a search based in squared meter.
#to improve, it is possible to get the next id recommendation in de soup

start = 0
finish = 1

list_loft_id = []


for num_neighborhood in neighborhood:
    #print(num_neighborhood)
    
    for x in range(1000):
        #print(start , finish)

        url = f"https://www.loft.com.br/apartamentos/sao-paulo-sp?neighbourhood=[{num_neighborhood}]&footage=[{start},{finish}]"
        response = requests.get(url)
        html = response.content
        soup = BeautifulSoup(html)

        start += 2
        finish += 2

        card_positions = len(soup.find_all('div', attrs={'data-testid': 'home-card'}))
        for card in range(card_positions):
            list_loft_id.append(soup.find_all('div', attrs={'data-testid': 'home-card'})[card].get('id'))

        #print(list_loft_id)start = 0
finish = 1

list_loft_id = []


for num_neighborhood in neighborhood:
    #print(num_neighborhood)
    
    for x in range(1000):
        #print(start , finish)

        url = f"https://www.loft.com.br/apartamentos/sao-paulo-sp?neighbourhood=[{num_neighborhood}]&footage=[{start},{finish}]"
        response = requests.get(url)
        html = response.content
        soup = BeautifulSoup(html)

        start += 2
        finish += 2

        card_positions = len(soup.find_all('div', attrs={'data-testid': 'home-card'}))
        for card in range(card_positions):
            list_loft_id.append(soup.find_all('div', attrs={'data-testid': 'home-card'})[card].get('id'))

        #print(list_loft_id)

In [ ]:
len(list_loft_id)

In [ ]:
df_loft_id = pd.DataFrame(list_loft_id)

In [ ]:
df_loft_id.to_csv(r'../data/raw-data/df_loft_id.csv', index=False)

##  scraping apartments

In [ ]:
df_loft_id = pd.read_csv(r'../data/raw-data/df_loft_id.csv')

In [ ]:
#this code extract the data apartment using the id previously collected
df_loft_apartament = pd.DataFrame()
for id_loft_ap in df_loft_id['0']:
    
    try:
        url_apartment_buy = f'https://www.loft.com.br/home/{id_loft_ap}'
        response = requests.get(url_apartment_buy)
        html = response.content
        soup = BeautifulSoup(html)


        data_apartment = json.loads(soup.find_all('script', attrs={'id':"__NEXT_DATA__"})[0].contents[0])['props']['pageProps']['home']
        df1 = pd.DataFrame.from_dict(data_apartment, orient='index').T
        df_loft_apartament = df_loft_apartament.append(df1, ignore_index=True)
    except:
        pass

In [ ]:
df_loft_apartament.columns

In [ ]:
df_loft_apartament2 = df_loft_apartament[['id', 'type', 'area', 'bedrooms', 'suits', 'parkingSpots', 'floor', 'complexFee', 'propertyTax', 'price',  'address', 'amenities','currentPhase']]

In [ ]:
addresses = df_loft_apartament2['address'].apply(lambda x : pd.Series(x))
df_loft_raw = pd.concat([df_loft_apartament2, addresses], axis=1)

In [ ]:
df_loft_raw['id_site'] = df_loft_raw.iloc[:,0] 

In [ ]:
df_loft_raw = df_loft_raw[['id_site', 'type', 'area', 'bedrooms', 'suits', 'parkingSpots', 'floor', 'complexFee', 'propertyTax', 'price', 'amenities', 'currentPhase','lat', 'lng', 'city', 'number', 'postalCode', 'streetName', 'streetType', 'unitNumber', 'neighborhood']]

In [ ]:
df_loft_raw.to_csv(r'../data/raw-data/df_loft_raw.csv', index=False)

# Google - palces

## loading api_key

In [22]:
#loading api_key
fkey = open('config.txt', 'r')
api_key = fkey.read()
gmaps = googlemaps.Client(key = api_key)

## selecting neighborhood

In [ ]:
df_loft_neigh = pd.read_csv(r'data/df_loft_raw.csv')

In [ ]:
#list of neighborhood to find geolocation
list_loft_neighborhood = []
for x in loft['bairro'].unique():
    list_loft_neighborhood.append(x)

In [ ]:
#busca a geolocalização do bairro para buscar os pontos dele. 
#Para a busca utilizei apenas o ponto central do bairro para agilizar a busca. Utilizei uma distância que englobaria praticamente todo bairro
bairro_latitude = {}
for bairro in lista_bairros_loft:
    try:
        lat_lgn_bairro = gmaps.geocode(address = f'bairro + {bairro} ')[0]['geometry']['location']
        bairro_latitude[bairro] = lat_lgn_bairro
    except:
        pass

In [ ]:
#crio um dataframe com o bairro e sua latitude e longitude
df1 = pd.DataFrame(bairro_latitude.values())
df2 = pd.DataFrame(bairro_latitude.keys())
df_bairros =pd.concat([df1, df2], axis=1)


In [ ]:
#agrupei os serviços para agilizar a quantidade de buscas
lugares_alimentacao = ['bakery', 'bar', 'cafe', 'restaurant']
lugares_servicos = ['bank','beauty_salon','dentist','gym','laundry','lawyer']
lugares_comercio = ['shopping','supermarket','loja de roupas','drugstore', 'florist','posto de gasolina','estacioanemtno','farmácia'  ]
lugares_cultura = ['museum']
lugares_saude = ['doctor','hospital']
lugares_educacao = ['library','primary_school','school','secondary_school','university']
lugares_transporte = ['bus_station']
lugares_religiosos = ['church', 'cemetery','synagogue']
lugares_governamentais = ['fire_station','police','post_office']
lugares_parques = ['park']
lugares_estadios = ['stadium']
lugares_metro_trem = ['estação de metrô','estação de trem']
lugares_turismo = ['tourist_attraction']

In [ ]:
#listo as categorias que vou buscar no places
lista_query = [lugares_alimentacao, lugares_servicos, lugares_comercio, lugares_cultura, lugares_saude, lugares_educacao,
               lugares_transporte, lugares_religiosos, lugares_governamentais, lugares_parques, lugares_estadios, 
               lugares_metro_trem, lugares_turismo]

In [ ]:

lista_locations = []
lista_locations = list(zip(df_bairros['lat'], df_bairros['lng']))
len(lista_locations)

In [ ]:
lugares_google = []
location = 0
for locations in lista_locations:
    location += 1 #para ver se o cogigo esta rodando
    print(location)
    for item_query in lista_query:

        params = {
            'query': item_query,
            'location': locations,
            'radius': 2000,
            'open_now' : False
        }


        x = gmaps.places(**params)

        lugares_google.append(x['results'])

        for pagina in range(2):
            try:
                params['page_token'] = x['next_page_token'] 
                time.sleep(2)
                x = gmaps.places(**params)
                lugares_google.append(x['results'])
            except:
                continue

In [ ]:
df_google_places2 = pd.DataFrame(lugares_google[0])
for lista_lugares in lugares_google:
    df_google_places2 = df_google_places2.append(lista_lugares)

In [ ]:
df_google_places2.reset_index(drop=True)

In [ ]:
df_google_places2.drop_duplicates(inplace=True)

In [ ]:
df_google_places2.to_csv(r'../data/raw-data/df_google_places2.csv', index=False)